In [1]:
import duckdb
import pandas as pd
import os
from create_database import (
    add_posts_table,
    add_comments_working_table,
    add_initial_comments_tables,
    add_comments_to_comments_tables,
    create_lookup_table,
    create_subreddit_tables,
    create_threads_table,
)
from stats import (
    create_row_counts_table,
    get_depth_distribution,
    get_number_of_threads,
    get_thread_score_distribution,
    get_subreddit_distribution,
    table_stats,
    calculate_weighted_average,
    get_thread_lengths,
    get_author_distribution,
)
from filter_database import make_threads_unique, filter_threads

if os.path.exists("../data/database_sample.db"):
    os.remove("../data/database_sample.db")
if os.path.exists("../data/saved_stats.json"):
    os.remove("../data/saved_stats.json")
con = duckdb.connect("../data/database_sample.db")

con = duckdb.connect("../database_backup.db")
# print all tables in the database
print(con.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall())
create_threads_table(con, "unfiltered_threads")

In [2]:
import duckdb
import pandas as pd
import os
from get_samples import get_random_thread_details


def save_thread_to_csv(thread_row, db_path):
    """
    Takes a thread row and saves its associated data to two CSV files:
    posts_samples.csv and comments_samples.csv

    Args:
        thread_row: A DataFrame returned from get_random_thread_details
        db_path: Path to the DuckDB database
    """
    con = duckdb.connect(db_path)

    try:
        # Convert thread_row to DataFrame if it isn't already
        if not isinstance(thread_row, pd.DataFrame):
            thread_row = pd.DataFrame([thread_row])

        # Get all columns from the thread
        columns = thread_row.columns.tolist()

        # Initialize DataFrames for posts and comments
        posts_data = None
        comments_data = []

        # Process each column
        for col in columns:
            id_value = thread_row[col].iloc[0]  # Get the first (and only) value

            # Skip if the ID is null or if it's not an ID column
            if pd.isna(id_value) or col in [
                "id",
                "created_utc",
                "score",
                "num_comments",
            ]:
                continue

            # Get the full row from the corresponding table
            try:
                row_data = con.execute(
                    f"SELECT * FROM {col} WHERE id = '{id_value}'"
                ).fetchdf()

                if not row_data.empty:
                    if col == "posts":
                        # Save posts data
                        posts_data = row_data
                    else:
                        # Accumulate comments data
                        comments_data.append(row_data)
            except Exception as e:
                print(f"Error processing {col} with ID {id_value}: {e}")

        # Save posts data if we have any
        if posts_data is not None:
            # Read existing data if file exists
            if os.path.exists("../data/posts_samples.csv"):
                existing_posts = pd.read_csv("../data/posts_samples.csv")
                posts_data = pd.concat([existing_posts, posts_data], ignore_index=True)

            # Write all data with headers
            posts_data.to_csv("../data/posts_samples.csv", index=False)

        # Save comments data if we have any
        if comments_data:
            # Concatenate all new comment rows
            new_comments = pd.concat(comments_data, ignore_index=True)

            # Read existing data if file exists
            if os.path.exists("../data/comments_samples.csv"):
                existing_comments = pd.read_csv("../data/comments_samples.csv")
                new_comments = pd.concat(
                    [existing_comments, new_comments], ignore_index=True
                )

            # Write all data with headers
            new_comments.to_csv("../data/comments_samples.csv", index=False)

    finally:
        con.close()

In [3]:
if os.path.exists("../data/database_sample.db"):
    os.remove("../data/database_sample.db")
if os.path.exists("../data/posts_samples.csv"):
    os.remove("../data/posts_samples.csv")
if os.path.exists("../data/comments_samples.csv"):
    os.remove("../data/comments_samples.csv")
if os.path.exists("../data/saved_stats.json"):
    os.remove("../data/saved_stats.json")

con = duckdb.connect("../database_backup.db")
for _ in range(3):
    thread = get_random_thread_details("unfiltered_threads", con)
    save_thread_to_csv(thread, "../database_backup.db")
"""for _ in range(1):
    thread = get_random_thread_details("threads", con)
    save_thread_to_csv(thread, "../database_backup.db")
for _ in range(4):
    thread = get_random_thread_details("filtered_threads", con)
    save_thread_to_csv(thread, "../database_backup.db")"""
con.close()

Random thread selected:
posts: gs8yjt
comments_to_posts: fs4weth


Full information for posts (ID: gs8yjt):
created_utc: 1590681760
id: gs8yjt
name: t3_gs8yjt
title: What's the most tech illiterate thing you've seen a person do?
selftext: NULL
subreddit: AskReddit
score: 9918
upvote_ratio: 0.96
num_comments: 5321
archived: True
author: Lo-Fi_Kuzco
distinguished: NULL
media: NULL

--------------------------------------------------------------------------------

Full information for comments_to_posts (ID: fs4weth):
created_utc: 1590703287
id: fs4weth
body:
    Maybe like seven years back when my mom was in her early 50s, she did not know you can open
    another tab on a browser. She would close Internet Explorer (oof) every time she finished one
    topic, then reopen it and search for the new. Sometimes she’d go to website a, close the
    browser, go to website b, close the browser, and head back to website a. It was unbelievable.

score: 2
author: edszebra22
parent_id: gs8yjt

------

In [23]:
# Add ab_ as a prefix to the column parent_id in the comments_samples.csv file
df = pd.read_csv("../data/comments_samples.csv")
df["parent_id"] = "ab_" + df["parent_id"].astype(str)
df.to_csv("../data/comments_samples.csv", index=False)

In [2]:
con = duckdb.connect("../data/database_sample.db")
add_posts_table(con, "../data/posts_samples.csv")
add_comments_working_table(con, "../data/comments_samples.csv")
add_initial_comments_tables(con)
add_comments_to_comments_tables(con)

posts table created successfully.
Batch 1: Inserted 60 rows (Total: 60)
Finished loading 60 total rows
Inserted matching comments into comments_to_posts table successfully.
Successfully moved and deleted matching comments
Inserted matching comments into comments_to_comments_1 table successfully.
Successfully moved and deleted matching comments
Found 7 comments for level 2
Created level 2 table and deleted processed rows
Found 5 comments for level 3
Created level 3 table and deleted processed rows
Found 5 comments for level 4
Created level 4 table and deleted processed rows
Found 5 comments for level 5
Created level 5 table and deleted processed rows
Found 3 comments for level 6
Created level 6 table and deleted processed rows
Found 3 comments for level 7
Created level 7 table and deleted processed rows
Found 3 comments for level 8
Created level 8 table and deleted processed rows
Found 3 comments for level 9
Created level 9 table and deleted processed rows
Found 2 comments for level 10


In [3]:
create_row_counts_table(con)
df = con.execute("SELECT * FROM row_counts").fetchdf()

df = df.sort_values(by="row_count", ascending=False)
# Pretty-print the DataFrame
print(df.to_string(index=False))

             table_name  row_count
                  posts         14
      comments_to_posts         14
 comments_to_comments_1         10
 comments_to_comments_2          7
 comments_to_comments_4          5
 comments_to_comments_3          5
 comments_to_comments_5          5
 comments_to_comments_7          3
 comments_to_comments_9          3
 comments_to_comments_6          3
 comments_to_comments_8          3
comments_to_comments_10          2


In [4]:
create_lookup_table(con)

Valid hierarchical tables: ['posts', 'comments_to_posts', 'comments_to_comments_1', 'comments_to_comments_2', 'comments_to_comments_3', 'comments_to_comments_4', 'comments_to_comments_5', 'comments_to_comments_6', 'comments_to_comments_7', 'comments_to_comments_8', 'comments_to_comments_9', 'comments_to_comments_10']
lookup_table created successfully.


In [5]:
table_stats("lookup_table", con)
calculate_weighted_average("thread_lengths_lookup_table")
calculate_weighted_average("thread_widths_lookup_table")
calculate_weighted_average("all_widths_lookup_table")

In [6]:
create_threads_table(con, "all_threads")
make_threads_unique(con, "threads")

In [7]:
filter_threads(con, "threads")

In [8]:
get_depth_distribution("threads", con)
get_depth_distribution("filtered_threads", con)

In [9]:
get_thread_lengths("threads", con)
get_thread_lengths("filtered_threads", con)

In [12]:
get_number_of_threads("threads", con)
get_number_of_threads("filtered_threads", con)

In [13]:
get_thread_score_distribution("threads", con)
get_thread_score_distribution("filtered_threads", con)

In [14]:
get_subreddit_distribution("threads", con)
get_subreddit_distribution("filtered_threads", con)

In [15]:
get_author_distribution("threads", con)
get_author_distribution("filtered_threads", con)

In [17]:
subreddits = ["AskReddit", "memes", "politics"]
for subreddit in subreddits:
    create_subreddit_tables(con, subreddit)
    table_stats(f"{subreddit}_lookup", con)
    calculate_weighted_average(f"thread_lengths_{subreddit}_lookup")
    calculate_weighted_average(f"thread_widths_{subreddit}_lookup")
    calculate_weighted_average(f"all_widths_{subreddit}_lookup")
    get_depth_distribution(f"{subreddit}_threads", con)
    get_depth_distribution(f"filtered_{subreddit}_threads", con)
    get_thread_lengths(f"{subreddit}_threads", con)
    get_thread_lengths(f"filtered_{subreddit}_threads", con)
    get_number_of_threads(f"{subreddit}_threads", con)
    get_number_of_threads(f"filtered_{subreddit}_threads", con)
    get_thread_score_distribution(f"{subreddit}_threads", con)
    get_thread_score_distribution(f"filtered_{subreddit}_threads", con)
    get_author_distribution(f"{subreddit}_threads", con)
    get_author_distribution(f"filtered_{subreddit}_threads", con)

No data found for thread_lengths_politics_lookup
No data found for thread_widths_politics_lookup
No data found for all_widths_politics_lookup


In [18]:
# Add up the row counts for all tables starting with "comments"
comments_tables = con.execute(
    "SELECT table_name FROM information_schema.tables WHERE table_name LIKE 'comments_%'"
).fetchdf()
total_comments = 0
for table in comments_tables["table_name"]:
    count = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    total_comments += count
print(f"Total number of comments across all tables: {total_comments}")

Total number of comments across all tables: 60


In [19]:
# Look at database_sample.db
for table in con.execute("SHOW TABLES").fetchdf()["name"]:
    print(f"Table: {table}")
    print(con.execute(f"SELECT * FROM {table}").fetchdf())
    print("\n")

Table: AskReddit_ids
       id
0  gbj767


Table: AskReddit_lookup
    posts comments_to_posts comments_to_comments_1 comments_to_comments_2  \
0  gbj767         [fp6o3g0]              [fp8jcvm]              [fp9l2d4]   

  comments_to_comments_3 comments_to_comments_4 comments_to_comments_5  \
0              [fp9m0mt]              [fpamy5s]              [fpaoglu]   

  comments_to_comments_6 comments_to_comments_7 comments_to_comments_8  \
0              [fpaq7zt]              [fpaqgwk]              [fpareob]   

  comments_to_comments_9 comments_to_comments_10  
0              [fpasr0k]               [fpatq96]  


Table: AskReddit_threads
    posts comments_to_posts comments_to_comments_1 comments_to_comments_2  \
0  gbj767           fp6o3g0                fp8jcvm                fp9l2d4   

  comments_to_comments_3 comments_to_comments_4 comments_to_comments_5  \
0                fp9m0mt                fpamy5s                fpaoglu   

  comments_to_comments_6 comments_to_comments_7

In [20]:
con.commit()
con.close()